# HexMagic

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *
from nbdev import nbdev_export

In [ ]:
nbdev_export()

How do I generate local documentation with nbdev

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

To generate local documentation with nbdev, use:

```bash
nbdev_docs
```

This will build the documentation site locally in the `_docs` directory. You can then preview it by opening `_docs/index.html` in your browser, or by serving it with a local web server:

```bash
python -m http.server --directory _docs
```

If you want the docs to automatically rebuild when you make changes, you can use:

```bash
nbdev_preview
```

This starts a local server (typically at `http://localhost:3000`) and watches for changes to your notebooks, rebuilding the docs automatically.

In [ ]:
#| hide
#import nbdev; nbdev.nbdev_export()
import sys
import math
from fastcore.basics import patch

#| export
## Introduction

The purpose of this library is to generate hex maps that are used in board games.

#| export
## Getting Started

In [ ]:
#| export
from HexMagic.primitives import Hex, MapCord 


In [ ]:
#| export
from HexMagic.styles import StyleCSS,  SVGBuilder

#| export
Hex, short for hexagon, is a six-sided polygon that can be laid out in a honeycomb pattern that is space-efficient and strong. It also has a nicer perimeter than a square. This library groups them in a hexgrid, a honeycomb type structure

In [ ]:
#| export
from HexMagic.primitives import HexGrid



In [ ]:
#| export
from HexMagic.primitives import HexPosition , MapPath


In [ ]:
#| export
from HexMagic.primitives import HexRegion

In [ ]:
from HexMagic.terrain import Terrain
from HexMagic.primitives import MapSize, MapRect, MapCord , GosperCurve
import numpy as np